In [1]:
import pandas as pd
import pyam
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import openpyxl
import aneris

In [2]:
image_path = '../../../data/TOD/model_results/raw/Consolidated_IMAGE_GEO7_R3_results_12022025_0956.xlsx'
mimosa_path = '../../../data/TOD/model_results/raw/MIMOSA_processed_2.xlsx'
gem_e3_path = '../../../data/TOD/model_results/raw/GEM_E3_v2.xlsx'

In [3]:
regions_un = [
    'Latin America and Caribbean (UN-R5)',
    'Western Europe and Other States (UN-R5)',
    'World', 'Africa (UN-R5)',
    'Asia and the Pacific (UN-R5)',
    'Eastern Europe (UN-R5)']

regions_chpt_19 = ['IMAGE 3.3|Canada',
            'IMAGE 3.3|Japan',
            'IMAGE 3.3|Mexico',
            'IMAGE 3.3|South Africa',
            'IMAGE 3.3|Turkey',
            'IMAGE 3.3|India',
            'IMAGE 3.3|Rest of Southern Africa',
            'IMAGE 3.3|Brazil',
            'IMAGE 3.3|China Region',
            'IMAGE 3.3|Eastern Africa',
            'IMAGE 3.3|Indonesia Region',
            'IMAGE 3.3|Northern Africa',
            'IMAGE 3.3|Rest of South Asia',
            'IMAGE 3.3|Russia Region',
            'IMAGE 3.3|USA',
            'IMAGE 3.3|Western Africa',
            'IMAGE 3.3|Western Europe',
            'IMAGE 3.3|Oceania']

df_image_all = pyam.IamDataFrame(image_path)

gdp_image = df_image_all.filter(variable=['GDP*'], year=[2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100])
# gdp_image_un = gdp_image.filter(region=regions_un, year=[2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100])
# gdp_image_19 = gdp_image.filter(region=regions_chpt_19, year=[2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100])
# gdp_image.variable

[INFO] 11:40:07 - pyam.core: Reading file ..\..\..\data\TOD\model_results\raw\Consolidated_IMAGE_GEO7_R3_results_12022025_0956.xlsx


In [4]:
def round_to_0_1_percent(value):
    if value == 0:
        return 0
    # Determine the number of significant digits to round to
    magnitude = np.floor(np.log10(abs(value))) - 2
    rounding_factor = 10 ** magnitude
    return np.round(value / rounding_factor) * rounding_factor

In [5]:
def compute_3_groups_image(df, variables, weight=None):
    df_high_income = df.aggregate_region(variables,
                              region='High-Income',
                              subregions=[
                                'CAN',
                                'JAP',
                                'USA',
                                'WEU',
                                'OCE'
                              ],
                              weight=weight
                              )

    df_middle_income = df.aggregate_region(variables,
                              region='Middle-Income',
                              subregions=[
                                'MEX',
                                'SAF',
                                'TUR',
                                'BRA',
                                'CHN',
                                'INDO',
                                'RUS',
                              ],
                              weight=weight
                              )

    df_low_income = df.aggregate_region(variables,
                              region='Low-Income',
                              subregions=[
                                'INDIA',
                                'RSAF',
                                'EAF',
                                'NAF',
                                'RSAS',
                                'WAF',
                              ],
                              weight=weight
                              )

    df_3_groups = pyam.concat([df_high_income, df_middle_income, df_low_income])
    return df_3_groups

def compute_UN_groups_image(df, variables, weight=None):
    df_africa = df.aggregate_region(variables,
                              region='Africa (UN-R5)',
                              subregions=[
                                    'NAF',
                                    'WAF',
                                    'EAF',
                                    'SAF',
                                    'RSAF',
                              ],
                              weight=weight
                              )

    df_asia = df.aggregate_region(variables,
                              region='Asia and the Pacific (UN-R5)',
                              subregions=[
                                'CHN',
                                'INDIA',
                                'INDO',
                                'JAP',
                                'KOR',
                                'ME',
                                'RSAS',
                                'SEAS',
                                'STAN',
                              ],
                              weight=weight
                              )

    df_ee = df.aggregate_region(variables,
                              region='Eastern Europe (UN-R5)',
                              subregions=[
                                    'CEU',
                                    'RUS',
                                    'UKR',
                              ],
                              weight=weight
                              )

    df_latin = df.aggregate_region(variables,
                              region='Latin America and Caribbean (UN-R5)',
                              subregions=[
                                    'BRA',
                                    'MEX',
                                    'RCAM',
                                    'RSAM',
                              ],
                              weight=weight
                              )

    df_we = df.aggregate_region(variables,
                              region='Western Europe and Other States (UN-R5)',
                              subregions=[
                                        'CAN',
                                        'OCE',
                                        'TUR',
                                        'USA',
                                        'WEU',
                              ],
                              weight=weight
                              )
    df_world = df.filter(variable=variables, region='World')

    df_un_groups = pyam.concat([df_africa, df_asia, df_ee, df_latin, df_we, df_world])
    return df_un_groups

def compute_3_groups_gem(df, variables, weight=None):
    df_high_income = df.aggregate_region(variables,
                              region='High-Income',
                              subregions=[
                                'USA',
                                'CAN',
                                'JPN',
                                'OCE',
                                'EU28'
                              ],
                              weight=weight
                              )

    df_middle_income = df.aggregate_region(variables,
                              region='Middle-Income',
                              subregions=[
                                'MEX',
                                'SAF',
                                'TUR',
                                'BRA',
                                'CHN',
                                'IDN',
                                'RUS',
                              ],
                              weight=weight
                              )

    df_low_income = df.aggregate_region(variables,
                              region='Low-Income',
                              subregions=[
                                'IND',
                                'ROW',
                              ],
                              weight=weight
                              )

    df_3_groups = pyam.concat([df_high_income, df_middle_income, df_low_income])
    return df_3_groups

def compute_UN_groups_gem(df, variables, weight=None):
    
    df_africa = df.aggregate_region(variables,
                              region='Africa (UN-R5)',
                              subregions=[
                                    'SAF', 
                                    'ROW'
                              ],
                              weight=weight
                              )

    df_asia = df.aggregate_region(variables,
                              region='Asia and the Pacific (UN-R5)',
                              subregions=[
                                'JPN', 
                                'CHN',
                                'IND',
                                'KOR',
                                'IDN',
                                'SAR',
                                'REP',
                              ],
                              weight=weight
                              )

    df_ee = df.aggregate_region(variables,
                              region='Eastern Europe (UN-R5)',
                              subregions=[
                                'REU',
                                'RUS',
                              ],
                              weight=weight
                              )

    df_latin = df.aggregate_region(variables,
                              region='Latin America and Caribbean (UN-R5)',
                              subregions=[
                                'ARG',
                                'BRA',
                                'MEX',
                              ],
                              weight=weight
                              )

    df_we = df.aggregate_region(variables,
                              region='Western Europe and Other States (UN-R5)',
                              subregions=[
                                'EU28',
                                'USA',
                                'CAN',
                                'TUR',
                                'OCE',
                              ],
                              weight=weight
                              )
    
    df_world = df.filter(variable=variables, region='World')

    df_un_groups = pyam.concat([df_africa, df_asia, df_ee, df_latin, df_we, df_world])
    return df_un_groups


# Calculate Convertion Ratio

In [6]:

gdp_image_un = compute_UN_groups_image(gdp_image, gdp_image.variable)
df_un_ratio = gdp_image_un.divide('GDP|PPP', 'GDP|MER', 'GDP|Ratio', ignore_units='')
df_un_ppp_input = gdp_image_un.multiply('GDP|MER', 1, 'GDP|MER|Input', ignore_units='billion USD_2010/yr')
df_un_image = pyam.concat([df_un_ppp_input, df_un_ratio])

df_3_groups_image = compute_3_groups_image(gdp_image, gdp_image.variable)
df_3_ratio = df_3_groups_image.divide('GDP|PPP', 'GDP|MER', 'GDP|Ratio', ignore_units='')
df_3_ppp_input = df_3_groups_image.multiply('GDP|MER', 1, 'GDP|MER|Input', ignore_units='billion USD_2010/yr')
df_3_image = pyam.concat([df_3_ppp_input, df_3_ratio])
df_3_image.region

['High-Income', 'Low-Income', 'Middle-Income']

# MIMOSA and GEM-E3 Income groups

In [7]:
df_unclean_mimosa = pd.read_excel('../../../data/TOD/model_results/raw/MIMOSA_processed_2.xlsx')
ratio_mask = df_unclean_mimosa['Unit'] == 'fraction of GDP'
# Apply rounding to the 'value' column for rows where the 'unit' column is '%'
# df_unclean_mimosa.loc[ratio_mask, df_unclean_mimosa.columns[5:]] = df_unclean_mimosa.loc[ratio_mask, df_unclean_mimosa.columns[5:]]  * 100
# df_unclean_mimosa.loc[ratio_mask, 'Unit'] = "% of GDP"

df_mid_clean_mimosa = pyam.IamDataFrame(df_unclean_mimosa)
df_mid_clean_mimosa.rename(scenario={
        "REF": "REF-v2",
        "LIFE-TP": "LIFE-TP-v2",
        "TECH-TP": "TECH-TP-v2",
    }, inplace=True)


df_un_mimosa = compute_UN_groups_image(df_mid_clean_mimosa, 'GDP|Climate Damage Corrected|Loss|Relative', weight='GDP|Climate Damage Corrected')
df_3_goups_mimosa = compute_3_groups_image(df_mid_clean_mimosa, 'GDP|Climate Damage Corrected|Loss|Relative', weight='GDP|Climate Damage Corrected')
df_3_goups_mimosa.timeseries()

2020  \
model  scenario   region        variable                                   unit                    
MIMOSA LIFE-TP-v2 High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
       REF-v2     High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
       TECH-TP-v2 High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   

                                                                                                2030  \
model  scenario   region        variable                                   unit                        
MIMOSA LIFE-TP-v2 High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.003938   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.015010   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.006037   
       REF-v2     High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.004683   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.018208   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.006948   
       TECH-TP-v2 High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.003921   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.014938   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.006016   

                                                                                                2040  \
model  scenario   region        variable                                   unit                        
MIMOSA LIFE-TP-v2 High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.007391   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.026827   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.011207   
       REF-v2     High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.011475   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.043599   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.015606   
       TECH-TP-v2 High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.007393   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.026844   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.011210   

                                                                                                2050  \
model  scenario   region        variable                                   unit                        
MIMOSA LIFE-TP-v2 High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.008042   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.026514   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.013275   
       REF-v2     High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.020172   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fr

In [8]:
df_unclean_gem = pd.read_excel('../../../data/TOD/model_results/raw/GEM_E3_v2.xlsx')
df_unclean_gem['Scenario'] = df_unclean_gem['Scenario'].replace({'DECARB_LIFE': 'LIFE-TP-v2',
                                                        'DECARB_TECH': 'TECH-TP-v2',
                                                        'REF_V2':'REF-v2',
                                                  })

df_mid_clean_gem = pyam.IamDataFrame(df_unclean_gem)
df_mid_clean_gem.filter(variable=["GDP|MER", 'Population'])
df_mid_clean_gem.rename(unit={'billion US$2010/yr':'billion USD_2010/yr'}, inplace=True)

df_un_gem = compute_UN_groups_gem(df_mid_clean_gem, ["GDP|MER", 'Population'])
df_3_groups_gem = compute_3_groups_gem(df_mid_clean_gem, ["GDP|MER", 'Population'])
df_3_groups_gem.timeseries()

2015  \
model        scenario   region        variable   unit                                
GEM-E3_V2023 LIFE-TP-v2 High-Income   GDP|MER    billion USD_2010/yr  41527.174647   
                                      Population million               1032.375424   
                        Low-Income    GDP|MER    billion USD_2010/yr   8947.068280   
                                      Population million               3409.710058   
                        Middle-Income GDP|MER    billion USD_2010/yr  17914.158252   
                                      Population million               2270.461976   
             REF-v2     High-Income   GDP|MER    billion USD_2010/yr  41527.174647   
                                      Population million               1032.375424   
                        Low-Income    GDP|MER    billion USD_2010/yr   8947.068280   
                                      Population million               3409.710058   
                        Middle-Income GDP|MER    billion USD_2010/yr  17914.158252   
                                      Population million               2270.461976   
             TECH-TP-v2 High-Income   GDP|MER    billion USD_2010/yr  41527.174647   
                                      Population million               1032.375424   
                        Low-Income    GDP|MER    billion USD_2010/yr   8947.068280   
                                      Population million               3409.710058   
                        Middle-Income GDP|MER    billion USD_2010/yr  17914.158252   
                                      Population million               2270.461976   

                                                                              2020  \
model        scenario   region        variable   unit                                
GEM-E3_V2023 LIFE-TP-v2 High-Income   GDP|MER    billion USD_2010/yr  42001.582509   
                                      Population million               1051.723709   
                        Low-Income    GDP|MER    billion USD_2010/yr  10261.152858   
                                      Population million               3684.994699   
                        Middle-Income GDP|MER    billion USD_2010/yr  21198.710837   
                                      Population million               2343.921774   
             REF-v2     High-Income   GDP|MER    billion USD_2010/yr  42001.582508   
                                      Population million               1051.723709   
                        Low-Income    GDP|MER    billion USD_2010/yr  10261.152884   
                                      Population million               3684.994699   
                        Middle-Income GDP|MER    billion USD_2010/yr  21198.710853   
                                      Population million               2343.921774   
             TECH-TP-v2 High-Income   GDP|MER    billion USD_2010/yr  42001.582510   
                                      Population million               1051.723709   
                        Low-Income    GDP|MER    billion USD_2010/yr  10261.152940   
                                      Population million               3684.994699   
                        Middle-Income GDP|MER    billion USD_2010/yr  21198.710855   
                                      Population million               2343.921774   

                                                                              2025  \
model        scenario   region        variable   unit                                
GEM-E3_V2023 LIFE-TP-v2 High-Income   GDP|MER    billion USD_2010/yr  47311.769790   
                                      Population million               1066.676412   
                        Low-Income    GDP|MER    billion USD_2010/yr  13270.101082   
                                      Population million               3955.578312   
                        Middle-Income GDP|MER    billion USD_2010/yr  26908.478088   
                                  

# GEM-E3 MER to PPP

In [9]:
# Original multipled by ratio to get PPP, now just 1 as we want MER
df_un_gem.rename(model={'GEM-E3_V2023':'Macro_Economy'}, inplace=True)
df_un_mimosa.rename(model={'MIMOSA':'Macro_Economy'}, inplace=True)
df_un_image.rename(model={'IMAGE 3.3':'Macro_Economy'}, inplace=True)

df_un = pyam.concat([df_un_gem, df_un_mimosa, df_un_image])
df_un = df_un.filter(year=[2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100])
df_un_to_harmonize = df_un.multiply('GDP|MER', 1, 'GDP|MER|Narrative Effects', ignore_units='billion USD_2010/yr')

df_un_hist = df_un.filter(variable='GDP|MER|Input')
df_un_hist.rename(variable={'GDP|MER|Input':'GDP|MER|Narrative Effects'}, inplace=True)
df_un_to_harmonize.filter(region='Africa*').timeseries()

2020  \
model         scenario   region         variable                  unit                               
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|MER|Narrative Effects billion USD_2010/yr  7640.112260   
              REF-v2     Africa (UN-R5) GDP|MER|Narrative Effects billion USD_2010/yr  7640.112288   
              TECH-TP-v2 Africa (UN-R5) GDP|MER|Narrative Effects billion USD_2010/yr  7640.112347   

                                                                                               2030  \
model         scenario   region         variable                  unit                                
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|MER|Narrative Effects billion USD_2010/yr  11501.602111   
              REF-v2     Africa (UN-R5) GDP|MER|Narrative Effects billion USD_2010/yr  11650.199221   
              TECH-TP-v2 Africa (UN-R5) GDP|MER|Narrative Effects billion USD_2010/yr  11496.842314   

                                                                                               2040  \
model         scenario   region         variable                  unit                                
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|MER|Narrative Effects billion USD_2010/yr  17714.501467   
              REF-v2     Africa (UN-R5) GDP|MER|Narrative Effects billion USD_2010/yr  18101.644113   
              TECH-TP-v2 Africa (UN-R5) GDP|MER|Narrative Effects billion USD_2010/yr  17701.814620   

                                                                                               2050  \
model         scenario   region         variable                  unit                                
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|MER|Narrative Effects billion USD_2010/yr  26687.757093   
              REF-v2     Africa (UN-R5) GDP|MER|Narrative Effects billion USD_2010/yr  27184.750199   
              TECH-TP-v2 Africa (UN-R5) GDP|MER|Narrative Effects billion USD_2010/yr  26752.640270   

                                                                                               2060  \
model         scenario   region         variable                  unit                                
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|MER|Narrative Effects billion USD_2010/yr  38906.256429   
              REF-v2     Africa (UN-R5) GDP|MER|Narrative Effects billion USD_2010/yr  39372.514493   
              TECH-TP-v2 Africa (UN-R5) GDP|MER|Narrative Effects billion USD_2010/yr  39054.772551   

                                                                                               2070  \
model         scenario   region         variable                  unit                                
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|MER|Narrative Effects billion USD_2010/yr  48942.209787   
              REF-v2     Africa (UN-R5) GDP|MER|Narrative Effects billion USD_2010/yr  49256.448218   
              TECH-TP-v2 Africa (UN-R5) GDP|MER|Narrative Effects billion USD_2010/yr  49249.852322   

                                                                                               2080  \
model         scenario   region         variable                  unit                                
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|MER|Narrative Effects billion USD_2010/yr  60621.857314   
              REF-v2     Africa (UN-R5) GDP|MER|Narrative Effects billion USD_2010/yr  60303.607101   
              TECH-TP-v2 Africa (UN-R5) GDP|MER|Narrative Effects billion USD_2010/yr  61059.699363   

                                                                                               2090  \
model         scenario   region         variable                  unit                                
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|MER|Narrative Effects billion USD_2010/yr  74664.041314   
              REF-v2     Africa (UN-R5) GDP|MER|Narrative Effects billion USD_2010/yr  73237.899142   
              TECH-TP-v2 Africa (UN-R5) GDP|MER|Narrative Effects billion U

In [10]:
df_3_groups_gem.rename(model={'GEM-E3_V2023':'Macro_Economy'}, inplace=True)
df_3_goups_mimosa.rename(model={'MIMOSA':'Macro_Economy'}, inplace=True)
df_3_image.rename(model={'IMAGE 3.3':'Macro_Economy'}, inplace=True)

df_3 = pyam.concat([df_3_groups_gem, df_3_goups_mimosa, df_3_image])
df_3 = df_3.filter(year=[2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100])
df_3_to_harmonize = df_3.multiply('GDP|MER', 1, 'GDP|MER|Narrative Effects', ignore_units='billion USD_2010/yr')
df_3_hist = df_3.filter(variable='GDP|MER|Input')
df_3_hist.rename(variable={'GDP|MER|Input':'GDP|MER|Narrative Effects'}, inplace=True)
df_3_to_harmonize.timeseries()

2020  \
model         scenario   region        variable                  unit                                
Macro_Economy LIFE-TP-v2 High-Income   GDP|MER|Narrative Effects billion USD_2010/yr  42001.582509   
                         Low-Income    GDP|MER|Narrative Effects billion USD_2010/yr  10261.152858   
                         Middle-Income GDP|MER|Narrative Effects billion USD_2010/yr  21198.710837   
              REF-v2     High-Income   GDP|MER|Narrative Effects billion USD_2010/yr  42001.582508   
                         Low-Income    GDP|MER|Narrative Effects billion USD_2010/yr  10261.152884   
                         Middle-Income GDP|MER|Narrative Effects billion USD_2010/yr  21198.710853   
              TECH-TP-v2 High-Income   GDP|MER|Narrative Effects billion USD_2010/yr  42001.582510   
                         Low-Income    GDP|MER|Narrative Effects billion USD_2010/yr  10261.152940   
                         Middle-Income GDP|MER|Narrative Effects billion USD_2010/yr  21198.710855   

                                                                                              2030  \
model         scenario   region        variable                  unit                                
Macro_Economy LIFE-TP-v2 High-Income   GDP|MER|Narrative Effects billion USD_2010/yr  51391.306040   
                         Low-Income    GDP|MER|Narrative Effects billion USD_2010/yr  16823.957896   
                         Middle-Income GDP|MER|Narrative Effects billion USD_2010/yr  32070.673639   
              REF-v2     High-Income   GDP|MER|Narrative Effects billion USD_2010/yr  51733.039360   
                         Low-Income    GDP|MER|Narrative Effects billion USD_2010/yr  17085.809109   
                         Middle-Income GDP|MER|Narrative Effects billion USD_2010/yr  32404.435616   
              TECH-TP-v2 High-Income   GDP|MER|Narrative Effects billion USD_2010/yr  51405.702565   
                         Low-Income    GDP|MER|Narrative Effects billion USD_2010/yr  16835.035033   
                         Middle-Income GDP|MER|Narrative Effects billion USD_2010/yr  32094.161199   

                                                                                              2040  \
model         scenario   region        variable                  unit                                
Macro_Economy LIFE-TP-v2 High-Income   GDP|MER|Narrative Effects billion USD_2010/yr  60766.762613   
                         Low-Income    GDP|MER|Narrative Effects billion USD_2010/yr  26278.481865   
                         Middle-Income GDP|MER|Narrative Effects billion USD_2010/yr  39460.235117   
              REF-v2     High-Income   GDP|MER|Narrative Effects billion USD_2010/yr  61480.604096   
                         Low-Income    GDP|MER|Narrative Effects billion USD_2010/yr  26980.537631   
                         Middle-Income GDP|MER|Narrative Effects billion USD_2010/yr  40621.378554   
              TECH-TP-v2 High-Income   GDP|MER|Narrative Effects billion USD_2010/yr  60835.323013   
                         Low-Income    GDP|MER|Narrative Effects billion USD_2010/yr  26295.611523   
                         Middle-Income GDP|MER|Narrative Effects billion USD_2010/yr  39580.197154   

                                                                                              2050  \
model         scenario   region        variable                  unit                                
Macro_Economy LIFE-TP-v2 High-Income   GDP|MER|Narrative Effects billion USD_2010/yr  72571.580599   
                         Low-Income    GDP|MER|Narrative Effects billion USD_2010/yr  39649.507191   
                         Middle-Income GDP|MER|Narrative Effects billion USD_2010/yr  46434.175029   
              REF-v2     High-Income   GDP|MER|Narrative Effects billion USD_2010/yr  73643.505062   
                         Low-Income    GDP|MER|Narrative Effects billion USD_2010/yr  40628.614221   
           

# Harmonize to SSP2 Input

In [11]:
# AIM will be harmonized to IMAGE REF historic values
def harmonize_df(df_gem, df_image_hist):

    # # Create a dataframe that contains the method of harmonization per variable
    # # Only create methods for variables present in AIM
    methods = df_gem.timeseries().reset_index(level='model', drop=True)
    # The offset methods ensure internal regional consistency better than the ratio methods
    methods['method'] = 'constant_offset' # 'reduce_ratio_2080' 'constant_ratio' 'constant_offset' 'reduce_offset_2080'
    methods_series = methods['method']

    hist_to_harmonize = df_image_hist.timeseries()
    df_gem_to_harmonize = df_gem.timeseries()

    # Remove the model level from index and column names should be strings
    df_gem_to_harmonize = df_gem_to_harmonize.reset_index(level='model', drop=True)
    df_gem_to_harmonize.columns = df_gem_to_harmonize.columns.astype(str)
    hist_to_harmonize = hist_to_harmonize.reset_index(level='model', drop=True)
    hist_to_harmonize.columns = hist_to_harmonize.columns.astype(str)

    # Harmonize 
    h = aneris.harmonize.Harmonizer(df_gem_to_harmonize, hist_to_harmonize, config={'harmonize_year': '2020'})
    harmonized = h.harmonize(methods_series) # methods_copy

    # Add all necessary index columns
    new_level = 'Macro_Economy'
    df_index = harmonized.index.to_frame()  # Convert the MultiIndex to a DataFrame
    df_index['model'] = new_level  # Add the new level
    df_index = df_index[['model', 'scenario', 'region', 'variable', 'unit']]
    new_index = pd.MultiIndex.from_frame(df_index)  # Create a new MultiIndex with the added level
    harmonized.index = new_index

    df_harmonized = pyam.IamDataFrame(harmonized)
    return df_harmonized

df_un_ppp_harmonized = harmonize_df(df_un_to_harmonize, df_un_hist)
df_3_ppp_harmonized = harmonize_df(df_3_to_harmonize, df_3_hist)

INFO:root:Harmonizing with constant_offset


INFO:root:Harmonizing with constant_offset


In [12]:
df_un_harmonized = pyam.concat([df_un, df_un_ppp_harmonized])

df_un_harmonized.subtract(1, 
               'GDP|Climate Damage Corrected|Loss|Relative', 
               'GDP|Climate Damage Corrected|Loss|Relative (Inverted)',
               ignore_units='',
               append=True 
              )
df_un_harmonized.multiply('GDP|MER|Narrative Effects', 
               'GDP|Climate Damage Corrected|Loss|Relative (Inverted)', 
               'GDP|MER|Narrative and Climate Effects', 
               ignore_units='billion USD_2010/yr', 
               append=True)
df_un_harmonized.filter(region='World*').timeseries()

2020  \
model         scenario   region variable                                           unit                                
Macro_Economy LIFE-TP-v2 World  GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP          0.000000   
                                GDP|Climate Damage Corrected|Loss|Relative (Inv...                          1.000000   
                                GDP|MER                                            billion USD_2010/yr  80110.288125   
                                GDP|MER|Input                                      billion USD_2010/yr  70183.415502   
                                GDP|MER|Narrative Effects                          billion USD_2010/yr  70183.415502   
                                GDP|MER|Narrative and Climate Effects              billion USD_2010/yr  70183.415502   
                                GDP|Ratio                                                                   1.460084   
                                Population                                         million               7795.136232   
              REF-v2     World  GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP          0.000000   
                                GDP|Climate Damage Corrected|Loss|Relative (Inv...                          1.000000   
                                GDP|MER                                            billion USD_2010/yr  80110.288172   
                                GDP|MER|Input                                      billion USD_2010/yr  70183.415502   
                                GDP|MER|Narrative Effects                          billion USD_2010/yr  70183.415502   
                                GDP|MER|Narrative and Climate Effects              billion USD_2010/yr  70183.415502   
                                GDP|Ratio                                                                   1.460084   
                                Population                                         million               7795.136232   
              TECH-TP-v2 World  GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP          0.000000   
                                GDP|Climate Damage Corrected|Loss|Relative (Inv...                          1.000000   
                                GDP|MER                                            billion USD_2010/yr  80110.288231   
                                GDP|MER|Input                                      billion USD_2010/yr  70183.415502   
                                GDP|MER|Narrative Effects                          billion USD_2010/yr  70183.415502   
                                GDP|MER|Narrative and Climate Effects              billion USD_2010/yr  70183.415502   
                                GDP|Ratio                                                                   1.460084   
                                Population                                         million               7795.136232   

                                                                                                                 2030  \
model         scenario   region variable                                           unit                                 
Macro_Economy LIFE-TP-v2 World  GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP           0.007138   
                                GDP|Climate Damage Corrected|Loss|Relative (Inv...                           0.992862   
                                GDP|MER                                            billion USD_2010/yr  109010.301617   
                                GDP|MER|Input                                      billion USD_2010/yr   97533.981353   
                                GDP|MER|Narrative Effects                          billion USD_2010/yr   99083.428994   
                                GDP|MER|Narrative and Climate Effects              billion USD_2010/yr   98376.172254   
                       

In [13]:
df_3_harmonized = pyam.concat([df_3, df_3_ppp_harmonized])

df_3_harmonized.subtract(1, 
               'GDP|Climate Damage Corrected|Loss|Relative', 
               'GDP|Climate Damage Corrected|Loss|Relative (Inverted)',
               ignore_units='',
               append=True 
              )
df_3_harmonized.multiply('GDP|MER|Narrative Effects', 
               'GDP|Climate Damage Corrected|Loss|Relative (Inverted)', 
               'GDP|MER|Narrative and Climate Effects', 
               ignore_units='billion USD_2010/yr', 
               append=True)

df_3_harmonized.filter(region='Low-Income').timeseries()

2020  \
model         scenario   region     variable                                           unit                                
Macro_Economy LIFE-TP-v2 Low-Income GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP          0.000000   
                                    GDP|Climate Damage Corrected|Loss|Relative (Inv...                          1.000000   
                                    GDP|MER                                            billion USD_2010/yr  10261.152858   
                                    GDP|MER|Input                                      billion USD_2010/yr   4703.359185   
                                    GDP|MER|Narrative Effects                          billion USD_2010/yr   4703.359185   
                                    GDP|MER|Narrative and Climate Effects              billion USD_2010/yr   4703.359185   
                                    GDP|Ratio                                                                   2.745050   
                                    Population                                         million               3684.994699   
              REF-v2     Low-Income GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP          0.000000   
                                    GDP|Climate Damage Corrected|Loss|Relative (Inv...                          1.000000   
                                    GDP|MER                                            billion USD_2010/yr  10261.152884   
                                    GDP|MER|Input                                      billion USD_2010/yr   4703.359185   
                                    GDP|MER|Narrative Effects                          billion USD_2010/yr   4703.359185   
                                    GDP|MER|Narrative and Climate Effects              billion USD_2010/yr   4703.359185   
                                    GDP|Ratio                                                                   2.745050   
                                    Population                                         million               3684.994699   
              TECH-TP-v2 Low-Income GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP          0.000000   
                                    GDP|Climate Damage Corrected|Loss|Relative (Inv...                          1.000000   
                                    GDP|MER                                            billion USD_2010/yr  10261.152940   
                                    GDP|MER|Input                                      billion USD_2010/yr   4703.359185   
                                    GDP|MER|Narrative Effects                          billion USD_2010/yr   4703.359185   
                                    GDP|MER|Narrative and Climate Effects              billion USD_2010/yr   4703.359185   
                                    GDP|Ratio                                                                   2.745050   
                                    Population                                         million               3684.994699   

                                                                                                                    2030  \
model         scenario   region     variable                                           unit                                
Macro_Economy LIFE-TP-v2 Low-Income GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP          0.015010   
                                    GDP|Climate Damage Corrected|Loss|Relative (Inv...                          0.984990   
                                    GDP|MER                                            billion USD_2010/yr  16823.957896   
                                    GDP|MER|Input                                      billion USD_2010/yr   8945.552051   
                                    GDP|MER|Narrative Effects                          billion USD_2010/yr  11266.164223   
                       

In [14]:
df_total = pyam.concat([df_un_harmonized, df_3_harmonized])
df_total.rename(scenario={
    'LIFE-TP-v2':'LIFE-TP',
    'REF-v2':'REF',
    'TECH-TP-v2':'TECH-TP'
}, inplace=True)

df_total.divide('GDP|MER|Input', "Population", 'GDP|MER|Input|Per Capita', ignore_units='thousand USD_2010/yr', append=True)
df_total.divide('GDP|MER|Narrative Effects', "Population", 'GDP|MER|Narrative Effects|Per Capita', ignore_units='thousand USD_2010/yr', append=True)
df_total.divide('GDP|MER|Narrative and Climate Effects', "Population", 'GDP|MER|Narrative and Climate Effects|Per Capita', ignore_units='thousand USD_2010/yr', append=True)

df_total = df_total.timeseries().reset_index()
df_total.loc[:, df_total.columns[5:]] = df_total.loc[:, df_total.columns[5:]].applymap(round_to_0_1_percent)


C:\Users\maycocks\AppData\Local\Temp\2\ipykernel_4604\2587054212.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_total.loc[:, df_total.columns[5:]] = df_total.loc[:, df_total.columns[5:]].applymap(round_to_0_1_percent)


In [15]:
df_total_un = df_total[df_total['region'].isin([
    'Africa (UN-R5)', 
    'Asia and the Pacific (UN-R5)',
    'Eastern Europe (UN-R5)',
    'Latin America and Caribbean (UN-R5)',
    'Western Europe and Other States (UN-R5)',
    'World'])]
df_total_un = df_total_un[df_total_un['variable'].isin([
    'GDP|MER|Input',
    # 'GDP|PPP|Narrative Effects',
    'GDP|MER|Narrative and Climate Effects',
    'GDP|MER|Input|Per Capita',
    # 'GDP|PPP|Narrative Effects|Per Capita',
    'GDP|MER|Narrative and Climate Effects|Per Capita'
    ])].reset_index(drop=True)
df_total_un.to_excel('../../../data/TOD/model_results/to_share/GDP_to_share_MER.xlsx', index=False)
# df_total_un.to_excel('../../../data/SOD/model_results/to_share/GDP_to_share_chapter_20.xlsx', index=False)
# df_total_un.to_excel('../../../data/SOD/model_results/to_share/GDP_to_share_chapter_14.xlsx', index=False)
df_total_un

,model,scenario,region,variable,unit,2020,2030,2040,2050,2060,2070,2080,2090,2100
0,Macro_Economy,LIFE-TP,Africa (UN-R5),GDP|MER|Input,billion USD_2010/yr,2420.00,3870.00,6880.00,12200.00,21600.00,36900.00,60200.0,93300.0,137000.0
1,Macro_Economy,LIFE-TP,Africa (UN-R5),GDP|MER|Input|Per Capita,thousand USD_2010/yr,1.02,1.39,2.13,3.37,5.38,8.47,13.0,19.2,27.2
2,Macro_Economy,LIFE-TP,Africa (UN-R5),GDP|MER|Narrative and Climate Effects,billion USD_2010/yr,2420.00,6200.00,12200.00,20900.00,33000.00,43000.00,54600.0,68600.0,84600.0
3,Macro_Economy,LIFE-TP,Africa (UN-R5),GDP|MER|Narrative and Climate Effects|Per Capita,thousand USD_2010/yr,1.02,2.23,3.77,5.77,8.22,9.88,11.8,14.1,16.8
4,Macro_Economy,LIFE-TP,Asia and the Pacific (UN-R5),GDP|MER|Input,billion USD_2010/yr,24700.00,39500.00,54700.00,70400.00,86200.00,103000.00,121000.0,139000.0,158000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Macro_Economy,TECH-TP,Western Europe and Other States (UN-R5),GDP|MER|Narrative and Climate Effects|Per Capita,thousand USD_2010/yr,35.80,43.10,49.80,59.70,71.80,81.60,92.1,103.0,115.0
68,Macro_Economy,TECH-TP,World,GDP|MER|Input,billion USD_2010/yr,70200.00,97500.00,127000.00,160000.00,198000.00,246000.00,303000.0,371000.0,452000.0
69,Macro_Economy,TECH-TP,World,GDP|MER|Input|Per Capita,thousand USD_2010/yr,9.00,11.40,13.70,16.40,19.50,23.40,28.3,34.1,41.3
70,Macro_Economy,TECH-TP,World,GDP|MER|Narrative and Climate Effects,billion USD_2010/yr,70200.00,98400.00,126000.00,160000.00,203000.00,240000.00,281000.0,327000.0,379000.0


In [16]:
df_total_income = df_total[df_total['region'].isin([
    'High-Income', 
    'Low-Income',
    'Middle-Income',
    ])]
df_total_income = df_total_income[df_total_income['variable'].isin([
    'GDP|MER|Input',
    # 'GDP|PPP|Narrative Effects',
    'GDP|MER|Narrative and Climate Effects',
    'GDP|MER|Input|Per Capita',
    # 'GDP|PPP|Narrative Effects|Per Capita',
    'GDP|MER|Narrative and Climate Effects|Per Capita'
    ])].reset_index(drop=True)
df_total_income.to_excel('../../../data/TOD/model_results/to_share/GDP_to_share_chapter_19_MER.xlsx', index=False)
df_total_income

,model,scenario,region,variable,unit,2020,2030,2040,2050,2060,2070,2080,2090,2100
0,Macro_Economy,LIFE-TP,High-Income,GDP|MER|Input,billion USD_2010/yr,40500.00,48900.00,56700.00,65100.00,74300.00,84700.00,95300.0,106000.0,117000.0
1,Macro_Economy,LIFE-TP,High-Income,GDP|MER|Input|Per Capita,thousand USD_2010/yr,38.50,45.30,51.50,58.80,66.90,75.90,84.9,93.7,102.0
2,Macro_Economy,LIFE-TP,High-Income,GDP|MER|Narrative and Climate Effects,billion USD_2010/yr,40500.00,49700.00,58800.00,70500.00,84900.00,96600.00,109000.0,123000.0,138000.0
3,Macro_Economy,LIFE-TP,High-Income,GDP|MER|Narrative and Climate Effects|Per Capita,thousand USD_2010/yr,38.50,46.00,53.40,63.70,76.40,86.60,97.4,109.0,121.0
4,Macro_Economy,LIFE-TP,Low-Income,GDP|MER|Input,billion USD_2010/yr,4700.00,8950.00,15600.00,26100.00,42100.00,65700.00,98600.0,142000.0,198000.0
5,Macro_Economy,LIFE-TP,Low-Income,GDP|MER|Input|Per Capita,thousand USD_2010/yr,1.28,2.12,3.29,5.03,7.54,11.10,16.1,22.6,30.9
6,Macro_Economy,LIFE-TP,Low-Income,GDP|MER|Narrative and Climate Effects,billion USD_2010/yr,4700.00,11100.00,20200.00,33200.00,50200.00,66100.00,85200.0,107000.0,133000.0
7,Macro_Economy,LIFE-TP,Low-Income,GDP|MER|Narrative and Climate Effects|Per Capita,thousand USD_2010/yr,1.28,2.63,4.24,6.39,9.00,11.20,13.9,17.0,20.7
8,Macro_Economy,LIFE-TP,Middle-Income,GDP|MER|Input,billion USD_2010/yr,16800.00,28000.00,38400.00,47800.00,55500.00,62600.00,68900.0,74700.0,80400.0
9,Macro_Economy,LIFE-TP,Middle-Income,GDP|MER|Input|Per Capita,thousand USD_2010/yr,7.18,11.50,15.70,19.70,23.50,27.60,31.6,35.8,40.2
